<a href="https://colab.research.google.com/github/mdivate/CNN-Transformer-and-Multidimensional-Attention-Mechanism/blob/main/Speech_Emotion_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall -y torch torchvision torchaudio
!pip install torch==2.9.0 torchvision==0.24.0 torchaudio==2.9.0 --index-url https://download.pytorch.org/whl/cu126

Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Looking in indexes: https://download.pytorch.org/whl/cu126
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 832.9/832.9 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 108.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 89.7 MB/s eta 0:00:00


In [ ]:
import torch
print(torch.__version__)
print("CUDA:", torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")



2.9.0+cu126
CUDA: True
Tesla T4


In [ ]:
!pip install -q librosa soundfile scikit-learn pandas
!pip install torch==2.9.0 torchvision==0.24.0 torchaudio==2.9.0 --index-url https://download.pytorch.org/whl/cu126

import torch
print("Torch:", torch.__version__)
print("CUDA:", torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


Looking in indexes: https://download.pytorch.org/whl/cu126
Torch: 2.9.0+cu126
CUDA: True
Tesla T4


In [ ]:
import os, glob, random
import numpy as np
import pandas as pd
import librosa
import soundfile as sf
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from google.colab import drive

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


In [ ]:
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
IEMOCAP_PATH = "/content/drive/MyDrive/IEMOCAP_full_release/"
EMO_LABELS = {"ang":0, "hap":1, "sad":2, "neu":3}


In [ ]:
import os
import glob
import re

iemocap_path = "/content/drive/MyDrive/IEMOCAP_full_release"
emotion_map = {}   # filename → emotion

# Regex matches exactly the format you pasted
line_regex = re.compile(
    r'^\[(.*?)\]\s+([A-Za-z0-9_]+)\s+([a-z]{3})\s+\[(.*?)\]'
)

label_files = glob.glob(iemocap_path + "/**/dialog/EmoEvaluation/*.txt", recursive=True)
print("Label files found:", len(label_files))

for lf in label_files:
    with open(lf, "r", errors="ignore") as f:
        for line in f:
            line = line.strip()
            match = line_regex.match(line)
            if match:
                filename = match.group(2) + ".wav"
                emotion = match.group(3).lower()

                # skip xxx, not labeled emotions
                if emotion == "xxx":
                    continue

                emotion_map[filename] = emotion

print("Total Labeled Files:", len(emotion_map))
print(list(emotion_map.items())[:10])


Label files found: 28
Total Labeled Files: 1404
[('Ses01F_script01_2_F003.wav', 'ang'), ('Ses01F_script01_2_F004.wav', 'ang'), ('Ses01F_script01_2_F005.wav', 'ang'), ('Ses01F_script01_2_F006.wav', 'ang'), ('Ses01F_script01_2_F007.wav', 'ang'), ('Ses01F_script01_2_F008.wav', 'ang'), ('Ses01F_script01_2_F009.wav', 'ang'), ('Ses01F_script01_2_F010.wav', 'ang'), ('Ses01F_script01_2_F011.wav', 'ang'), ('Ses01F_script01_2_F012.wav', 'ang')]


In [ ]:
import os
print(os.listdir("/content/drive/MyDrive/IEMOCAP_full_release/"))


['Session1']


In [ ]:
wav_files = glob.glob(IEMOCAP_PATH + "**/*.wav", recursive=True)
print("WAV files found:", len(wav_files))


WAV files found: 114


In [ ]:
data = []

for wav in wav_files:
    key = wav.split("/")[-1]
    if key in emotion_map:
        label_str = emotion_map[key]
        label_idx = EMO_LABELS.index(label_str)  # get integer index
        data.append((wav, label_idx))

print("Final usable dataset:", len(data))


Final usable dataset: 52


In [ ]:
!pip uninstall -y torchaudio
!pip install torchaudio==2.8.1

ERROR: Could not find a version that satisfies the requirement torchaudio==2.8.1 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0, 2.9.1)
ERROR: No matching distribution found for torchaudio==2.8.1


In [ ]:
import glob, os

iemocap_path = "/content/drive/MyDrive/IEMOCAP_full_release/"

# Use the emotions we want
EMO_LABELS = ["ang", "hap", "sad", "neu", "exc"]

emotion_map = {}

label_files = glob.glob(os.path.join(iemocap_path, "*/dialog/EmoEvaluation/*.txt"))

for lf in label_files:
    with open(lf, "r") as f:
        lines = f.readlines()
        current_wav = None
        current_emotions = []
        for line in lines:
            line = line.strip()
            if line.startswith("[") and "\t" in line:
                # New segment
                parts = line.split("\t")
                if len(parts) >= 2:
                    current_wav = parts[1]  # e.g., Ses01F_script01_2_F000
                    current_emotions = []
            elif line.startswith("C-") and ":" in line and current_wav:
                # Extract annotator emotion
                emo = line.split(":")[1].strip().lower().rstrip(";")
                # Map IEMOCAP labels to simple labels
                if emo.startswith("anger"):
                    emo_simple = "ang"
                elif emo.startswith("happiness") or emo.startswith("excited"):
                    emo_simple = "hap"
                elif emo.startswith("sadness"):
                    emo_simple = "sad"
                elif emo.startswith("neutral"):
                    emo_simple = "neu"
                elif emo.startswith("excited"):
                    emo_simple = "exc"
                else:
                    emo_simple = None

                if emo_simple in EMO_LABELS:
                    current_emotions.append(emo_simple)
            elif line == "" and current_wav:
                # End of segment, take first emotion if exists
                if current_emotions:
                    emotion_map[current_wav] = current_emotions[0]
                current_wav = None


In [ ]:
wav_files = glob.glob(os.path.join(iemocap_path, "**/*.wav"), recursive=True)
data = []

for wav in wav_files:
    key = os.path.basename(wav).replace(".wav","")
    if key in emotion_map:
        data.append((wav, emotion_map[key]))

print("Final usable dataset:", len(data))


Final usable dataset: 85


In [ ]:
!pip uninstall -y torch torchaudio


Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126


In [ ]:
!pip install torch==2.2.0+cu118 torchaudio==2.2.0+cu118 --extra-index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118


In [ ]:
import torch
import torchaudio

print(torch.__version__)
print(torchaudio.__version__)
print(torchaudio.get_audio_backend())



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.12/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.12/dist-package

2.2.0+cu118
2.2.0+cu118
None


/tmp/ipython-input-3249698272.py:6: UserWarning: torchaudio._backend.get_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  print(torchaudio.get_audio_backend())


In [ ]:
import glob, os

# Path to your IEMOCAP dataset
iemocap_path = "/content/drive/MyDrive/IEMOCAP_full_release/"

# Emotions we care about
EMO_LABELS = ["ang", "hap", "sad", "neu", "exc"]

# Build emotion map from EmoEvaluation files
emotion_map = {}
label_files = glob.glob(os.path.join(iemocap_path, "*/dialog/EmoEvaluation/*.txt"))

for lf in label_files:
    with open(lf, "r") as f:
        lines = f.readlines()
        current_wav = None
        current_emotions = []
        for line in lines:
            line = line.strip()
            if line.startswith("[") and "\t" in line:
                parts = line.split("\t")
                if len(parts) >= 2:
                    current_wav = parts[1]  # file ID
                    current_emotions = []
            elif line.startswith("C-") and ":" in line and current_wav:
                emo = line.split(":")[1].strip().lower().rstrip(";")
                if emo.startswith("anger"):
                    emo_simple = "ang"
                elif emo.startswith("happiness") or emo.startswith("excited"):
                    emo_simple = "hap"
                elif emo.startswith("sadness"):
                    emo_simple = "sad"
                elif emo.startswith("neutral"):
                    emo_simple = "neu"
                elif emo.startswith("excited"):
                    emo_simple = "exc"
                else:
                    emo_simple = None
                if emo_simple in EMO_LABELS:
                    current_emotions.append(emo_simple)
            elif line == "" and current_wav:
                if current_emotions:
                    emotion_map[current_wav] = current_emotions[0]
                current_wav = None

# Match WAV files to labels
wav_files = glob.glob(os.path.join(iemocap_path, "**/*.wav"), recursive=True)
data = []
for wav in wav_files:
    key = os.path.basename(wav).replace(".wav","")
    if key in emotion_map:
        data.append((wav, emotion_map[key]))

print("Final usable dataset:", len(data))
print("First 5 entries:", data[:5])


Final usable dataset: 85
First 5 entries: [('/content/drive/MyDrive/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script03_2_F013.wav', 'ang'), ('/content/drive/MyDrive/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script03_2_F033.wav', 'ang'), ('/content/drive/MyDrive/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script03_2_F025.wav', 'ang'), ('/content/drive/MyDrive/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script03_2_F019.wav', 'ang'), ('/content/drive/MyDrive/IEMOCAP_full_release/Session1/sentences/wav/Ses01M_script03_2_F037.wav', 'ang')]


In [ ]:
import torchaudio

# Check available backends
print("Available torchaudio backends:", torchaudio.list_audio_backends())

# Set the backend explicitly to soundfile
torchaudio.set_audio_backend("soundfile")

# Load the first WAV
wav_path = data[0][0]
waveform, sample_rate = torchaudio.load(wav_path)
print("Waveform shape:", waveform.shape)
print("Sample rate:", sample_rate)


Available torchaudio backends: ['ffmpeg', 'soundfile']


/tmp/ipython-input-3584490978.py:7: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")


Waveform shape: torch.Size([1, 97600])
Sample rate: 16000


In [ ]:
# -----------------------------
# 1️⃣ Imports
# -----------------------------
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchaudio
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import os

# -----------------------------
# 2️⃣ Use the data list you already created
# data = [(wav_path, label_str), ...] with 85 samples
# -----------------------------
labels = [label for _, label in data]
le = LabelEncoder()
le.fit(labels)
num_classes = len(le.classes_)

# -----------------------------
# 3️⃣ PyTorch Dataset
# -----------------------------
class SERDataset(Dataset):
    def __init__(self, data, label_encoder, n_mfcc=40, max_len=200):
        """
        Args:
            data: list of tuples (wav_path, label_str)
            label_encoder: sklearn LabelEncoder fitted on labels
            n_mfcc: number of MFCC features
            max_len: maximum number of time steps (for padding/truncation)
        """
        self.data = data
        self.le = label_encoder
        self.n_mfcc = n_mfcc
        self.max_len = max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        wav_path, label_str = self.data[idx]

        # Load waveform using soundfile backend
        waveform, sample_rate = torchaudio.load(wav_path)

        # Convert to mono if needed
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)

        # Extract MFCC features
        mfcc = torchaudio.transforms.MFCC(
            sample_rate=sample_rate,
            n_mfcc=self.n_mfcc,
            melkwargs={'n_mels': 40, 'n_fft': 400, 'hop_length': 160}
        )(waveform)  # shape: [1, n_mfcc, time]

        mfcc = mfcc.squeeze(0).T  # shape: [time, n_mfcc]

        # Pad or truncate to fixed length
        if mfcc.shape[0] < self.max_len:
            pad = torch.zeros(self.max_len - mfcc.shape[0], self.n_mfcc)
            mfcc = torch.cat([mfcc, pad], dim=0)
        else:
            mfcc = mfcc[:self.max_len, :]

        # Convert label to tensor
        label = torch.tensor(self.le.transform([label_str])[0], dtype=torch.long)

        return mfcc.float(), label
# -----------------------------
# 4️⃣ Train/Test Split
# -----------------------------
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
train_dataset = SERDataset(train_data, le)
test_dataset = SERDataset(test_data, le)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# -----------------------------
# 5️⃣ CNN-BiLSTM Model
# -----------------------------
class CNN_BiLSTM(nn.Module):
    def __init__(self, n_mfcc=40, hidden_dim=128, num_layers=1, num_classes=num_classes):
        super(CNN_BiLSTM, self).__init__()
        self.conv1 = nn.Conv1d(n_mfcc, 64, kernel_size=5, padding=2)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv1d(64, 128, kernel_size=5, padding=2)
        self.lstm = nn.LSTM(input_size=128, hidden_size=hidden_dim, num_layers=num_layers,
                            batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_dim*2, num_classes)

    def forward(self, x):
        x = x.permute(0, 2, 1)  # [batch, n_mfcc, time]
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = x.permute(0, 2, 1)  # [batch, time, features]
        lstm_out, _ = self.lstm(x)
        out = self.fc(lstm_out[:, -1, :])  # take last timestep
        return out

# -----------------------------
# 6️⃣ Training Setup
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN_BiLSTM().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 20

# -----------------------------
# 7️⃣ Training Loop
# -----------------------------
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for mfccs, labels_batch in train_loader:
        mfccs, labels_batch = mfccs.to(device), labels_batch.to(device)
        optimizer.zero_grad()
        outputs = model(mfccs)
        loss = criterion(outputs, labels_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * mfccs.size(0)
        _, predicted = torch.max(outputs, 1)
        total += labels_batch.size(0)
        correct += (predicted == labels_batch).sum().item()
    train_loss = running_loss / total
    train_acc = correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {train_loss:.4f} - Acc: {train_acc:.4f}")

# -----------------------------
# 8️⃣ Evaluation
# -----------------------------
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for mfccs, labels_batch in test_loader:
        mfccs, labels_batch = mfccs.to(device), labels_batch.to(device)
        outputs = model(mfccs)
        _, predicted = torch.max(outputs, 1)
        total += labels_batch.size(0)
        correct += (predicted == labels_batch).sum().item()
test_acc = correct / total
print("Test Accuracy:", test_acc)


Epoch 1/20 - Loss: 0.8705 - Acc: 0.6618
Epoch 2/20 - Loss: 0.7447 - Acc: 0.7353
Epoch 3/20 - Loss: 0.6769 - Acc: 0.7353
Epoch 4/20 - Loss: 0.6584 - Acc: 0.7500
Epoch 5/20 - Loss: 0.5146 - Acc: 0.7941
Epoch 6/20 - Loss: 0.4596 - Acc: 0.8529
Epoch 7/20 - Loss: 0.3729 - Acc: 0.8824
Epoch 8/20 - Loss: 0.3109 - Acc: 0.8824
Epoch 9/20 - Loss: 0.3347 - Acc: 0.8824
Epoch 10/20 - Loss: 0.2706 - Acc: 0.8824
Epoch 11/20 - Loss: 0.2758 - Acc: 0.8971
Epoch 12/20 - Loss: 0.2437 - Acc: 0.8971
Epoch 13/20 - Loss: 0.1752 - Acc: 0.9412
Epoch 14/20 - Loss: 0.1724 - Acc: 0.9265
Epoch 15/20 - Loss: 0.1208 - Acc: 0.9706
Epoch 16/20 - Loss: 0.1021 - Acc: 0.9706
Epoch 17/20 - Loss: 0.0779 - Acc: 0.9853
Epoch 18/20 - Loss: 0.0738 - Acc: 0.9706
Epoch 19/20 - Loss: 0.0891 - Acc: 0.9559
Epoch 20/20 - Loss: 0.0656 - Acc: 0.9853
Test Accuracy: 0.5882352941176471
